In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('mixtral_output.csv')
df.head()

,Category,Cleaned_Resume,Predicted_Category
0,Data Science,Skills Programming Languages Python pandas num...,Data Science
1,Data Science,Education Details May 2013 May 2017 B E UIT RG...,Data Science
2,Data Science,Areas Interest Deep Learning Control System De...,Data Science
3,Data Science,Skills R Python SAP HANA Tableau SAP HANA SQL ...,Data Science
4,Data Science,Education Details MCA YMCAUST Faridabad Haryan...,Data Science


In [6]:
df[df.Category != df.Predicted_Category][['Category', 'Predicted_Category']].sample(10).to_csv('kritika_example.csv', index= False)

In [31]:
sample_data = df[df.Category != df.Predicted_Category].reset_index(drop = True)
sample_data.head()

,Category,Cleaned_Resume,Predicted_Category
0,Data Science,Skills R Python SAP HANA Tableau SAP HANA SQL ...,NaN
1,HR,KEY SKILLS Computerized accounting tally Since...,Operations Manager
2,HR,Computer Knowledge Proficient basic use MS off...,The resume states that the person has experien...
3,HR,SOFTWARE SKILLS General Computer Proficiency P...,Java Developer
4,HR,KEY SKILLS Computerized accounting tally Since...,Operations Manager


In [32]:
job_title = sample_data['Category']
predicted_job_title = sample_data['Predicted_Category']


In [38]:
sample_data.Predicted_Category.fillna("", inplace =True)

In [ ]:
together_api_key = "64abd49981caf177302a7812bb5c2dee937424ab78f7a277d71a608d3c07316c"

In [39]:
import openai
import pandas as pd

# Set your Together AI API key
together_api_key = together_api_key

# Initialize Together AI client
client = openai.OpenAI(api_key=together_api_key, base_url="https://api.together.xyz/v1")


def predict_job_titles(job_title, predicted_job_title,job_titles):
    predicted_titles = []
    context_data = "You are a skilled HR professional who is designated to resolve mismatch \
            in original and predicted job title. For e.g if job_title = 'Data Science' and \
            predicted_job_title = 'Data Scientist', then since both of them are similar, \
            you should return 'Data Science'. Similarly, if job_title = 'Mechanical Engineer'\
            and predicted_job_title = 'Mechanical Designer Engineer', you should return 'Mechanical Engineer' "
    for idx in range(0, len(sample_data)):
        print(f"Processing title {idx + 1}")
        
        prompt = f"Please compare job_title : {job_title[idx]} and predicted job title : {predicted_job_title[idx]}. \
            If the predicted job title is similar, then please predict the corresponding original original job title."
        # Check if predicted_job_title is similar to job_title
        if predicted_job_title[idx].lower() == job_title[idx].lower():
            classified_job_title = job_title[idx]  # Use original job_title
        else:
            # Send the request to the model
            response = client.chat.completions.create(
                messages=[{"role": "system", "content": context_data}, {"role": "user", "content": prompt}],
                model="mistralai/Mixtral-8x7B-Instruct-v0.1",
                max_tokens=50,
                temperature=0.1
            )

            # Extracting output
            output = response.choices[0].message.content.strip()
            # Extracting job title from the output
            output_job_title = output.split(":")[-1].strip()
            # Filtering the output based on provided job titles
            classified_job_title = [title for title in job_titles if title.lower() in 
                                    output_job_title.lower()] or [output_job_title]
            classified_job_title = classified_job_title[0]
            #print("Predicted Job Title:", classified_job_title)

        predicted_titles.append(classified_job_title)
    
    return predicted_titles


In [40]:
# Get unique job titles
job_titles = df['Category'].unique()

# Predict job titles for resumes
predicted_job_titles = predict_job_titles(sample_data['Category'] , sample_data['Predicted_Category'],job_titles)

# Print predicted job titles
print("Predicted Job Titles:", predicted_job_titles)

Processing title 1
Processing title 2
Processing title 3
Processing title 4
Processing title 5
Processing title 6
Processing title 7
Processing title 8
Processing title 9
Processing title 10
Processing title 11
Processing title 12
Processing title 13
Processing title 14
Processing title 15
Processing title 16
Processing title 17
Processing title 18
Processing title 19
Processing title 20
Processing title 21
Processing title 22
Processing title 23
Processing title 24
Processing title 25
Processing title 26
Processing title 27
Processing title 28
Processing title 29
Processing title 30
Processing title 31
Processing title 32
Processing title 33
Processing title 34
Processing title 35
Processing title 36
Processing title 37
Processing title 38
Processing title 39
Processing title 40
Processing title 41
Processing title 42
Processing title 43
Processing title 44
Processing title 45
Processing title 46
Processing title 47
Processing title 48
Processing title 49
Processing title 50
Processin

In [43]:
sample_data['LLM_Category'] = predicted_job_titles

In [44]:
sample_data

,Category,Cleaned_Resume,Predicted_Category,LLM_Category
0,Data Science,Skills R Python SAP HANA Tableau SAP HANA SQL ...,,"Based on the information provided, the predict..."
1,HR,KEY SKILLS Computerized accounting tally Since...,Operations Manager,HR
2,HR,Computer Knowledge Proficient basic use MS off...,The resume states that the person has experien...,"Based on the information provided, the predict..."
3,HR,SOFTWARE SKILLS General Computer Proficiency P...,Java Developer,HR
4,HR,KEY SKILLS Computerized accounting tally Since...,Operations Manager,HR
...,...,...,...,...
164,Testing,COMPUTER SKILLS SOFTWARE KNOWLEDGE MS Power Po...,Electrical Engineering,Electrical Engineering
165,Testing,COMPUTER PROFICIENCY Basic MS Office PowerPoin...,Automation Testing,Automation Testing
166,Testing,COMPUTER SKILLS SOFTWARE KNOWLEDGE MS Power Po...,"Based on the provided resume, the job title is...",Testing
167,Testing,COMPUTER PROFICIENCY Basic MS Office PowerPoin...,DotNet Developer,DotNet Developer


In [46]:
sample_data[sample_data.Category != sample_data.LLM_Category].shape

(40, 4)

In [47]:
df.shape

(962, 3)

In [49]:
1- (40/962)

0.9584199584199584

### GPT output

In [50]:
import pandas as pd
df = pd.read_csv('gpt_final_data_1.csv')
df.head()

,Category,Cleaned_Resume,Classfied_Job_Title
0,HR,TECHNICAL SKILLS Typewriting TORA SPSSEducatio...,HR
1,HR,Skills Windows XP Ms Office Word Excel Look up...,HR
2,HR,Education Details BA mumbai University HR Skil...,HR
3,HR,Education Details June 2012 May 2015 B Economi...,HR
4,HR,Education Details June 2012 May 2015 B Economi...,HR


In [67]:
df[df.Category != df.Classfied_Job_Title]

,Category,Cleaned_Resume,Classfied_Job_Title
10,HR,SOFTWARE SKILLS General Computer Proficiency P...,Java Developer
21,HR,SOFTWARE SKILLS General Computer Proficiency P...,Java Developer
32,HR,SOFTWARE SKILLS General Computer Proficiency P...,Java Developer
43,HR,SOFTWARE SKILLS General Computer Proficiency P...,Java Developer
64,Arts,Good communication skill Quick learner Keen fi...,Sales
68,Arts,Education Details August 2018 January 2021 Ent...,Health and fitness
70,Arts,Good communication skill Quick learner Keen fi...,HR
74,Arts,Education Details August 2018 January 2021 Ent...,Health and fitness
76,Arts,Good communication skill Quick learner Keen fi...,HR
80,Arts,Education Details August 2018 January 2021 Ent...,Health and fitness


In [69]:
1- 17/440

0.9613636363636364

In [80]:
1 - (37/962)

0.9615384615384616

In [74]:

sample_data = df[df.Category != df.Classfied_Job_Title].reset_index(drop = True)
sample_data.head()
job_title = sample_data['Category']
predicted_job_title = sample_data['Classfied_Job_Title']

In [75]:
sample_data.Classfied_Job_Title.fillna("", inplace =True)

In [76]:


import openai
import pandas as pd

# Set your Together AI API key
together_api_key = "64abd49981caf177302a7812bb5c2dee937424ab78f7a277d71a608d3c07316c"

# Initialize Together AI client
client = openai.OpenAI(api_key=together_api_key, base_url="https://api.together.xyz/v1")


def predict_job_titles(job_title, predicted_job_title,job_titles):
    predicted_titles = []
    context_data = "You are a skilled HR professional who is designated to resolve mismatch in original and predicted job title. For e.g if job_title = 'Data Science' and predicted_job_title = 'Data Scientist', then since both of them are similar, you should return 'Data Science'. Similarly, if job_title = 'Mechanical Engineer' and predicted_job_title = 'Mechanical Designer Engineer', you should return 'Mechanical Engineer' "
    for idx in range(0, len(sample_data)):
        print(f"Processing title {idx + 1}")
        
        prompt = f"Please compare job_title : {job_title[idx]} and predicted job title : {predicted_job_title[idx]}. If the predicted job title is similar, then please predict the corresponding original original job title."
        # Check if predicted_job_title is similar to job_title
        if predicted_job_title[idx].lower() == job_title[idx].lower():
            classified_job_title = job_title[idx]  # Use original job_title
        else:
            # Send the request to the model
            response = client.chat.completions.create(
                messages=[{"role": "system", "content": context_data}, {"role": "user", "content": prompt}],
                model="mistralai/Mixtral-8x7B-Instruct-v0.1",
                max_tokens=50,
                temperature=0.1
            )

            # Extracting output
            output = response.choices[0].message.content.strip()
            # Extracting job title from the output
            output_job_title = output.split(":")[-1].strip()
            # Filtering the output based on provided job titles
            classified_job_title = [title for title in job_titles if title.lower() in output_job_title.lower()] or [output_job_title]
            classified_job_title = classified_job_title[0]
            #print("Predicted Job Title:", classified_job_title)

        predicted_titles.append(classified_job_title)
    
    return predicted_titles

# Get unique job titles
job_titles = df['Category'].unique()

# Predict job titles for resumes
predicted_job_titles = predict_job_titles(sample_data['Category'] , sample_data['Classfied_Job_Title'],job_titles)

# Print predicted job titles
print("Predicted Job Titles:", predicted_job_titles)



Processing title 1
Processing title 2
Processing title 3
Processing title 4
Processing title 5
Processing title 6
Processing title 7
Processing title 8
Processing title 9
Processing title 10
Processing title 11
Processing title 12
Processing title 13
Processing title 14
Processing title 15
Processing title 16
Processing title 17
Processing title 18
Processing title 19
Processing title 20
Processing title 21
Processing title 22
Processing title 23
Processing title 24
Processing title 25
Processing title 26
Processing title 27
Processing title 28
Processing title 29
Processing title 30
Processing title 31
Processing title 32
Processing title 33
Processing title 34
Processing title 35
Processing title 36
Processing title 37
Processing title 38
Processing title 39
Processing title 40
Processing title 41
Processing title 42
Processing title 43
Processing title 44
Processing title 45
Processing title 46
Processing title 47
Processing title 48
Processing title 49
Processing title 50
Processin

In [77]:
sample_data['LLM_Category'] = predicted_job_titles
sample_data


,Category,Cleaned_Resume,Classfied_Job_Title,LLM_Category
0,SAP Developer,Education Details January 2016 Bachelor Engine...,"Based on the provided resume, the predicted jo...",SAP Developer
1,SAP Developer,Skills ETL Data Warehousing SQL PL SQL Basic C...,ETL Developer,SAP Developer
2,SAP Developer,Education Details July 2008 February 2012 COMP...,"Based on the provided resume, the predicted jo...",SAP Developer
3,SAP Developer,Skills ETL Data Warehousing SQL PL SQL Basic C...,ETL Developer,SAP Developer
4,SAP Developer,Education Details July 2008 February 2012 COMP...,"Based on the provided resume, the predicted jo...",SAP Developer
5,SAP Developer,Education Details May 2013 Master Computer App...,"Based on the resume provided, the predicted jo...",SAP Developer
6,SAP Developer,Education Details January 2016 Bachelor Engine...,"Based on the provided resume, the predicted jo...",SAP Developer
7,Automation Testing,Excellent grasping power learning new concepts...,Testing,Automation Testing
8,Automation Testing,SOCIAL SKILLS Ability establish trust work tea...,Electrical Engineering,Automation Testing
9,Automation Testing,TECHNICAL SKILLS Automation Testing Selenium W...,Testing,Automation Testing


In [79]:
sample_data[sample_data.Category != sample_data.LLM_Category].shape

(20, 4)

In [70]:
import pandas as pd
df = pd.read_csv('gpt_final_data_2.csv')
df.head()

,Category,Cleaned_Resume,Classfied_Job_Title
0,SAP Developer,Education Details May 2013 Master Computer App...,SAP Developer
1,SAP Developer,Education Details January 2016 Bachelor Engine...,"Based on the provided resume, the predicted jo..."
2,SAP Developer,Education Details SAP Technical Architect SAP ...,SAP Developer
3,SAP Developer,Skills ETL Data Warehousing SQL PL SQL Basic C...,ETL Developer
4,SAP Developer,Competencies SAP Business Intelligence Version...,SAP Developer


In [73]:
df[df.Category != df.Classfied_Job_Title]

,Category,Cleaned_Resume,Classfied_Job_Title
1,SAP Developer,Education Details January 2016 Bachelor Engine...,"Based on the provided resume, the predicted jo..."
3,SAP Developer,Skills ETL Data Warehousing SQL PL SQL Basic C...,ETL Developer
5,SAP Developer,Education Details July 2008 February 2012 COMP...,"Based on the provided resume, the predicted jo..."
9,SAP Developer,Skills ETL Data Warehousing SQL PL SQL Basic C...,ETL Developer
11,SAP Developer,Education Details July 2008 February 2012 COMP...,"Based on the provided resume, the predicted jo..."
12,SAP Developer,Education Details May 2013 Master Computer App...,"Based on the resume provided, the predicted jo..."
13,SAP Developer,Education Details January 2016 Bachelor Engine...,"Based on the provided resume, the predicted jo..."
15,Automation Testing,Excellent grasping power learning new concepts...,Testing
16,Automation Testing,SOCIAL SKILLS Ability establish trust work tea...,Electrical Engineering
17,Automation Testing,TECHNICAL SKILLS Automation Testing Selenium W...,Testing


In [ ]:
1- 17/440

0.9613636363636364